In [1]:
import pandas as pd
import pathlib
import glob
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.optimizers import RMSprop, Adam
from keras.layers import Dense,Flatten , Conv2D , MaxPooling2D , Dropout, Rescaling
from keras.models import Model , Sequential
from keras.callbacks import EarlyStopping
from keras.utils import image_dataset_from_directory

In [ ]:
data_dir = './kaggle/' # (본인 자동차 이미지 폴더 경로)

train_ds = image_dataset_from_directory( data_dir, validation_split=0.2, subset='training', labels='inferred',
                                                               label_mode='categorical', image_size=[224, 224], seed=123, interpolation='nearest', batch_size=64, shuffle=True )

val_ds = image_dataset_from_directory( data_dir, validation_split=0.2, subset='validation', labels='inferred',
                                                             label_mode='categorical', image_size=[224, 224], seed=123, interpolation='nearest', batch_size=64, shuffle=False )

def convert_to_float(image, label):
  image = tf.image.convert_image_dtype(image, dtype=tf.float32)
  return image, label

AUTOTUNE = tf.data.experimental.AUTOTUNE 
train_ds = (train_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE)) 
val_ds = (val_ds.map(convert_to_float).cache().prefetch(buffer_size=AUTOTUNE))

from keras.applications import DenseNet121
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224,224,3))

num = 1
for layer in base_model.layers:
    if num >= 144:
      layer.trainable = True
    else:
      layer.trainable = False
    num += 1

model = Sequential()

model.add(tf.keras.layers.experimental.preprocessing.Rescaling(1./255, input_shape=(224,224,3)))

model.add(base_model)

model.add(Conv2D(16, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(padding='same'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(33 , activation = 'softmax'))

model.compile(loss=tf.keras.losses.categorical_crossentropy,optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),metrics=['accuracy'])

history = model.fit(train_ds,batch_size=64,epochs=40,validation_data=val_ds)


In [ ]:
model.evaluate(train_ds), model.evaluate(val_ds)